**IMPORT KAGGLE LIBRARY USING PIP INSTALL KAGGLE**


In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/




*   GO TO KAGGLE
*   GO TO PROFILE
*   DOWNLOAD NEW API TOKEN
*   COME TO COLAB
*   CLICK ON FILES MENU
*   CLICK ON ADD NEW FILE
*   ADD THE API TOKEN WHICH WILL BE IN JSON FORMAT HERE
*   AFTER ADDING THE JSON FILE TYPE THE BELOW CODE ND RUN









In [ ]:
from google.colab import files
files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shrirajthakare","key":"cc461c38313bcf11ff0e4f4072b93fa2"}'}

In [ ]:
!ls /content/


 drive	'kaggle (1).json'   sample_data


In [ ]:
!mkdir -p /root/.kaggle
!cp /content/kaggle\(1\).json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
%cd "/content/drive/MyDrive/Kaggle"

/content/drive/MyDrive/Kaggle


In [ ]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

 87% 129M/149M [00:01<00:00, 64.8MB/s]
100% 149M/149M [00:01<00:00, 85.9MB/s]


In [ ]:
!ls#The ls command is used to list the files and directories in the current working directory.

/bin/bash: ls#The: command not found


In [ ]:
from zipfile import ZipFile#the data has been downloaded nd mostly it will be downloaded in ur drive->mydrive->kaggle location so go there copy path and paste it here
dataset="/content/drive/MyDrive/Kaggle/brain-tumor-mri-dataset.zip"#after getting the path extract the zipfile
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("dataset extracted")

dataset extracted


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import decode_predictions
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow import keras
import tensorflow as tf

In [ ]:
import keras.utils as image
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image


In [ ]:
w,h=299,299 #the original inception v3 was trained on images of size 299x299 hence we will also use the same

In [ ]:
model_pretrained=InceptionV3(weights="imagenet",include_top=True,input_shape=(w,h,3)) #And make an object of inception v3 which has arguments bearing weights,whether to include or drop layers and input shape of the images.The include_top=True argument means that the model includes the fully connected layer at the top of the network, which is used for classification. If include_top=False, the fully connected layer is not included, and the model can be used as a feature extractor.
#The weights argument specifies the weight initialization for the model. In this case, we are using the pre-trained weights provided by the ImageNet dataset. These weights have been learned by the InceptionV3 model during training on the ImageNet dataset

96112376/96112376 [==============================] - 3s 0us/step


In [ ]:
train_ds = keras.utils.image_dataset_from_directory(#Create an image dataset i.e it takes all images and converts them to numpy arrays
    directory = '/content/drive/MyDrive/Kaggle/Training',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(299,299)
)

Found 7690 files belonging to 5 classes.


In [ ]:

# Set the path to your training dataset

# Loop through the first 5 categories in your training dataset
for i, category_folder in enumerate(os.listdir(path)[:5]):
    category_path = os.path.join(path, category_folder)

    # Loop through the first 5 images in the category folder
    for j, image_filename in enumerate(os.listdir(category_path)[:5]):
        image_path = os.path.join(category_path, image_filename)

        # Open the image using PIL
        image = Image.open(image_path)

        # Display the image using the default image viewer
        image.show()


NotADirectoryError: ignored

In [ ]:
def process(image,label):#Rescaling images so that the pixel value will be bw 0 nd 1
    image = tf.cast(image/299. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/Kaggle/Training/limited_images/pituitary')))

437


In [ ]:

path = '/content/drive/MyDrive/Kaggle/Training/limited_images/meningioma'
for file_name in os.listdir(path):
    file_path = os.path.join(path, file_name)
    with Image.open(file_path) as img:
        img.show()
        break


In [ ]:
reduced_ds = keras.utils.image_dataset_from_directory(#Create an image dataset i.e it takes all images and converts them to numpy arrays
    directory = '/content/drive/MyDrive/Kaggle/Training/limited_images',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(299,299)
)

Found 1978 files belonging to 5 classes.


In [ ]:
cnn_feature = model_pretrained.predict(reduced_ds,verbose=0)


In [ ]:
preds=decode_predictions(cnn_feature)

35363/35363 [==============================] - 0s 0us/step


In [ ]:
# Create a new directory in the specified path
!mkdir -p /content/drive/MyDrive/Kaggle/Testing/limited_images

In [ ]:
import os
import random
import shutil

# Set the path to the training directory
testing_dir = '/content/drive/MyDrive/Kaggle/Testing'

# Set the path to the reduced images directory
reduced_images_dir = os.path.join(testing_dir, 'limited_images')


# Define the percentage of images to keep (40% in this case)
keep_percentage = 0.3

# Loop over the subdirectories (categories) in the training directory
for category in os.listdir(testing_dir):
    category_dir = os.path.join(testing_dir, category)

    # Check if the subdirectory is a directory (not a file)
    if os.path.isdir(category_dir):

        # Create a subdirectory in the reduced images directory with the same name
        reduced_category_dir = os.path.join(reduced_images_dir, category)
        os.mkdir(reduced_category_dir)

        # Get a list of all image files in the subdirectory
        image_files = [f for f in os.listdir(category_dir) if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')]

        # Calculate the number of images to keep for this category
        num_images = len(image_files)
        num_keep = int(num_images * keep_percentage)

        # Randomly select which images to keep
        keep_indices = random.sample(range(num_images), num_keep)

        # Loop over the image files and copy the selected ones to the reduced category directory
        for i, image_file in enumerate(image_files):
            if i in keep_indices:
                src_path = os.path.join(category_dir, image_file)
                dst_path = os.path.join(reduced_category_dir, image_file)
                shutil.copy(src_path, dst_path)


FileExistsError: ignored

In [ ]:
test_ds= keras.utils.image_dataset_from_directory(#Create an image dataset i.e it takes all images and converts them to numpy arrays
    directory = '/content/drive/MyDrive/Kaggle/Testing/limited_images',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(299,299)

)

Found 387 files belonging to 5 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Model

x = model_pretrained.layers[-2].output
output = Dense(10, activation='softmax')(x)

# Create a new model with the modified output layer
model = Model(inputs=model_pretrained.input, outputs=output)
output_layer = model.get_layer('dense')
output_layer = model.layers[-1]

model_pretrained.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


ValueError: ignored

In [ ]:
model_pretrained.fit(reduced_ds, epochs=10, batch_size=32,validation_data=test_ds)


Epoch 1/10


ValueError: ignored